In [1]:
from glob import glob
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 5)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
import cv2
import os, sys, time, datetime, random, math

In [3]:
!mkdir -p /tmp/empty

In [ ]:

!rsync -a --delete /tmp/empty/   /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/


In [4]:
!rsync -a --delete /tmp/empty/   /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/frames/

In [5]:
video_path = '/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/dataset/ufc/'
bugs_path = '/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/dataset/bugs/'

In [6]:
training_path = video_path + 'training/'
testing_path = video_path + 'testing/'

# use pandas to read folder of videos
training_videos = glob(training_path + '*.mp4')
training_videos = sorted(training_videos, key=lambda x: int(x.split('Videos')[-1].split('_')[0]))
training_videos = pd.DataFrame(training_videos, columns=['video'])

testing_videos = glob(testing_path + '*//' + '*.mp4')
testing_videos = sorted(testing_videos, key=lambda x: int(x[-7:-4]))
testing_videos = pd.DataFrame(testing_videos, columns=['video'])
testing_videos['file_name'] = testing_videos['video'].apply(lambda x: x.split('/')[-1])


# parse list (by lines) of video frame labels from MNAD/dataset/ufc/Temporal_Anomaly_Annotation_for_Testing_Videos.txt
testing_labels = pd.read_csv(video_path + 'Temporal_Anomaly_Annotation_for_Testing_Videos.txt', sep=' ', header=None)
testing_videos

,video,file_name
0,/home/smoothjazzuser/Desktop/videogame-anomoly...,Fighting044_x264.mp4
1,/home/smoothjazzuser/Desktop/videogame-anomoly...,Fighting024_x264.mp4
...,...,...
948,/home/smoothjazzuser/Desktop/videogame-anomoly...,Stealing063_x264.mp4
949,/home/smoothjazzuser/Desktop/videogame-anomoly...,Stealing043_x264.mp4


In [7]:
testing_labels.columns = ['file_name', 'catagory', 'start1', 'end1', 'start2', 'end2']
#replace all -1 with None
testing_labels = testing_labels.replace(-1, None)
testing_labels.head(5)

,file_name,catagory,start1,end1,start2,end2
0,Abuse028_x264.mp4,Abuse,165,240,None,None
1,Abuse030_x264.mp4,Abuse,1275,1360,None,None
2,Arrest001_x264.mp4,Arrest,1185,1485,None,None
3,Arrest007_x264.mp4,Arrest,1530,2160,None,None
4,Arrest024_x264.mp4,Arrest,1005,3105,None,None


In [8]:
# join testing_labels to testing_videos by file_name
testing_videos = testing_videos.merge(testing_labels, on='file_name', how='left')
# drop files that are not in testing_labels
testing_videos = testing_videos.dropna(subset=['catagory'])
# convert start1 and end1 to frame index range
testing_videos['anomaly_frames'] = testing_videos.apply(lambda x: list(range(int(x['start1']), int(x['end1']))), axis=1)
del testing_videos['start1']
del testing_videos['end1']
# convert start1 and end1 to frame index range if values are not None
testing_videos['anomaly_frames2'] = testing_videos.apply(lambda x: list(range(int(x['start2']), int(x['end2']))) if not x['start2'] == None else None, axis=1)
del testing_videos['start2']
del testing_videos['end2']
#combine all intefers from anomaly_frames and anomaly_frames2
testing_videos['anomaly_frames'] = testing_videos.apply(lambda x: x['anomaly_frames'] + x['anomaly_frames2'] if not x['anomaly_frames2'] == None else x['anomaly_frames'], axis=1)
del testing_videos['anomaly_frames2']
#sort index
testing_videos = testing_videos.sort_values(by=['file_name'])
#set integer index
testing_videos = testing_videos.reset_index(drop=True)
#replace filename with integer index, and then use zfill to pad with zeros
testing_videos['file_name'] = testing_videos.index
testing_videos['file_name'] = testing_videos['file_name'].apply(lambda x: str(x + 1).zfill(2))

testing_videos

,video,file_name,catagory,anomaly_frames
0,/home/smoothjazzuser/Desktop/videogame-anomoly...,01,Abuse,"[165, 166, 167, 168, 169, 170, 171, 172, 173, ..."
1,/home/smoothjazzuser/Desktop/videogame-anomoly...,02,Abuse,"[1275, 1276, 1277, 1278, 1279, 1280, 1281, 128..."
...,...,...,...,...
138,/home/smoothjazzuser/Desktop/videogame-anomoly...,139,Vandalism,"[1830, 1831, 1832, 1833, 1834, 1835, 1836, 183..."
139,/home/smoothjazzuser/Desktop/videogame-anomoly...,140,Vandalism,"[540, 541, 542, 543, 544, 545, 546, 547, 548, ..."


In [9]:
# index training
training_videos['file_name'] = training_videos.index
training_videos['file_name'] = training_videos['file_name'].apply(lambda x: str(x + 1).zfill(2))

training_videos

,video,file_name
0,/home/smoothjazzuser/Desktop/videogame-anomoly...,01
1,/home/smoothjazzuser/Desktop/videogame-anomoly...,02
...,...,...
798,/home/smoothjazzuser/Desktop/videogame-anomoly...,799
799,/home/smoothjazzuser/Desktop/videogame-anomoly...,800


In [10]:
# calculate output folder and name for each video based on file_name
testing_videos['output_folder'] = testing_videos['file_name'].apply(lambda x: bugs_path + 'symlink_loc/frames/' + x + '/')
training_videos['output_folder'] = training_videos['file_name'].apply(lambda x: bugs_path + 'training/frames/' + x + '/')

# create output folders
testing_videos['output_folder'].apply(lambda x: os.makedirs(x, exist_ok=True))
training_videos['output_folder'].apply(lambda x: os.makedirs(x, exist_ok=True))
str(testing_videos['output_folder'][0])

'/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/frames/01/'

In [11]:
# use ffmpeg to load video at 30fps and save frames to output folder
def save_frames(video_path, output_folder):
    # use 96% quality to save space
    #filenames start at 000
    os.system('ffmpeg -i ' + video_path + ' -vf fps=30 ' + output_folder + '%3d.jpg' + ' -q:v 96')
    # rename jpg to start at 00

testing_videos['video'].apply(lambda x: save_frames(x, str(testing_videos['output_folder'][testing_videos['video'] == x].values[0])))


ffmpeg version 5.1.1-1ubuntu1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 12 (Ubuntu 12.2.0-1ubuntu1)
  configuration: --prefix=/usr --extra-version=1ubuntu1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librist --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtheora --enable-libtwolame --enable-libvidstab --

0      None
1      None
       ... 
138    None
139    None
Name: video, Length: 140, dtype: object

In [10]:

training_videos['video'].apply(lambda x: save_frames(x, str(training_videos['output_folder'][training_videos['video'] == x].values[0])))

frame=27100 fps=2194 q=24.8 Lsize=N/A time=00:15:03.33 bitrate=N/A speed=73.1x    
video:224636kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
ffmpeg version 5.1.1-1ubuntu1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 12 (Ubuntu 12.2.0-1ubuntu1)
  configuration: --prefix=/usr --extra-version=1ubuntu1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librist --enable-librubberband -

0      None
1      None
       ... 
798    None
799    None
Name: video, Length: 800, dtype: object

In [12]:
# calculate which frames are anomalies
def range_anom(output_folder, anomaly_frames):
    files = glob(output_folder + '*.jpg')
    anom = []
    for i in range(len(files)):
        if i in anomaly_frames:
            anom.append(1)
        else:
            anom.append(0)

    return anom

testing_videos['anomaly'] = testing_videos.apply(lambda x: range_anom(x['output_folder'], x['anomaly_frames']), axis=1)

In [13]:
testing_videos

,video,file_name,catagory,anomaly_frames,output_folder,anomaly
0,/home/smoothjazzuser/Desktop/videogame-anomoly...,01,Abuse,"[165, 166, 167, 168, 169, 170, 171, 172, 173, ...",/home/smoothjazzuser/Desktop/videogame-anomoly...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,/home/smoothjazzuser/Desktop/videogame-anomoly...,02,Abuse,"[1275, 1276, 1277, 1278, 1279, 1280, 1281, 128...",/home/smoothjazzuser/Desktop/videogame-anomoly...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...
138,/home/smoothjazzuser/Desktop/videogame-anomoly...,139,Vandalism,"[1830, 1831, 1832, 1833, 1834, 1835, 1836, 183...",/home/smoothjazzuser/Desktop/videogame-anomoly...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
139,/home/smoothjazzuser/Desktop/videogame-anomoly...,140,Vandalism,"[540, 541, 542, 543, 544, 545, 546, 547, 548, ...",/home/smoothjazzuser/Desktop/videogame-anomoly...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [14]:
#concatenate all anomaly lists into one list
frame_labels_anomalies = []
for i in range(len(testing_videos)):
    frame_labels_anomalies = frame_labels_anomalies + testing_videos['anomaly'][i]

# export the list as a .npy file called /home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/Gall/frame_labels_bugs.npy
np.save(bugs_path + 'symlink_loc/GTall/frame_labels_bugs.npy', np.expand_dims(frame_labels_anomalies, axis = 0))

#export to csv
testing_videos.to_csv(bugs_path + 'testing_frame_labels_bugs.csv')
training_videos.to_csv(bugs_path + 'training_frame_labels_bugs.csv')

In [15]:
# for list sub dirs in /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/

sub = glob('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/*/001.jpg')
# for i in sub, copy to 000.jpg
for i in sub:
    os.system('cp ' + i + ' ' + i[:-7] + '000.jpg')

['/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/290/001.jpg',
 '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/684/001.jpg',
 '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/199/001.jpg',
 '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/130/001.jpg',
 '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/11/001.jpg',
 '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/12/001.jpg',
 '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/175/001.jpg',
 '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/252/001.jpg',
 '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/485/001.jpg',
 '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/97/001.jpg',
 '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/88/001.jpg',
 '/home/smoothjazzuser/vi

In [15]:
sub = glob('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/frames/*/001.jpg')
for i in sub:
    os.system('cp ' + i + ' ' + i[:-7] + '000.jpg')